# APRIORI for Action Rules

### Data

In [1]:
import pandas as pd
transactions = {'Sex': ['M', 'F', 'M', 'M', 'F', 'M', 'F'], 
                'Age': ['Y', 'Y', 'O', 'Y', 'Y', 'O', 'Y'],
                'Class': [1, 1, 2, 2, 1, 1, 2],
                'Embarked': ['S', 'C', 'S', 'C', 'S', 'C', 'C'],
                'Survived': [1, 1, 0, 0, 1, 1, 0],
               }
data = pd.DataFrame.from_dict(transactions)
data

,Sex,Age,Class,Embarked,Survived
0,M,Y,1,S,1
1,F,Y,1,C,1
2,M,O,2,S,0
3,M,Y,2,C,0
4,F,Y,1,S,1
5,M,O,1,C,1
6,F,Y,2,C,0


One Hot Encoding

In [2]:
#data = pd.get_dummies(data, sparse=True, columns=data.columns, prefix_sep='_<item>_')
data = pd.get_dummies(data, sparse=False, columns=data.columns, prefix_sep='_<item>_')
data

,Sex_<item>_F,Sex_<item>_M,Age_<item>_O,Age_<item>_Y,Class_<item>_1,Class_<item>_2,Embarked_<item>_C,Embarked_<item>_S,Survived_<item>_0,Survived_<item>_1
0,0,1,0,1,1,0,0,1,0,1
1,1,0,0,1,1,0,1,0,0,1
2,0,1,1,0,0,1,0,1,1,0
3,0,1,0,1,0,1,1,0,1,0
4,1,0,0,1,1,0,0,1,0,1
5,0,1,1,0,1,0,1,0,0,1
6,1,0,0,1,0,1,1,0,1,0


### Input parameters

In [3]:
stable_attributes = ['Sex','Age']
flexible_attributes = ['Class','Embarked']
target = 'Survived'
wanted_change_in_target = [0, 1]
min_stable_attributes = 2
min_flexible_attributes = 1 #min 1
min_unwanted_support = 1
min_unwanted_confidence = 0.5 #min 0.5
min_wanted_support = 2
min_wanted_confidence = 0.5 #min 0.5

### Bindings
Bind item columns with attributes

In [4]:
from collections import defaultdict

stable_items_binding = defaultdict(lambda: [])
flexible_items_binding = defaultdict(lambda: [])
target_items_binding = defaultdict(lambda: [])

for col in data.columns:
    is_continue = False
    # stable
    for attribute in stable_attributes:
        if col.startswith(attribute+'_<item>_'):
            stable_items_binding[attribute].append(col)
            is_continue = True
            break
    if is_continue is True:
        continue
    # flexible    
    for attribute in flexible_attributes:
        if col.startswith(attribute+'_<item>_'):
            flexible_items_binding[attribute].append(col)
            is_continue = True
            break
    if is_continue is True:
        continue
    # target    
    if col.startswith(target+'_<item>_'):
        target_items_binding[target].append(col)      

In [5]:
stable_items_binding  

defaultdict(<function __main__.<lambda>()>,
            {'Sex': ['Sex_<item>_F', 'Sex_<item>_M'],
             'Age': ['Age_<item>_O', 'Age_<item>_Y']})

In [6]:
flexible_items_binding 

defaultdict(<function __main__.<lambda>()>,
            {'Class': ['Class_<item>_1', 'Class_<item>_2'],
             'Embarked': ['Embarked_<item>_C', 'Embarked_<item>_S']})

In [7]:
target_items_binding 

defaultdict(<function __main__.<lambda>()>,
            {'Survived': ['Survived_<item>_0', 'Survived_<item>_1']})

### Stop List
The items from the same attribute can not be in the same itemset

In [8]:
stop_list = []
for items in stable_items_binding.values():
    stop_list.append(tuple(items))
for items in flexible_items_binding.values():
    stop_list.append(tuple(items))
for items in target_items_binding.values():
    stop_list.append(tuple(items))

### Split table
Split based on the target

In [9]:
frames = {}
for item in target_items_binding[target]:
    mask = data[item]==1
    frames[item] = data[mask]
frames

{'Survived_<item>_0':    Sex_<item>_F  Sex_<item>_M  Age_<item>_O  Age_<item>_Y  Class_<item>_1  \
 2             0             1             1             0               0   
 3             0             1             0             1               0   
 6             1             0             0             1               0   
 
    Class_<item>_2  Embarked_<item>_C  Embarked_<item>_S  Survived_<item>_0  \
 2               1                  0                  1                  1   
 3               1                  1                  0                  1   
 6               1                  1                  0                  1   
 
    Survived_<item>_1  
 2                  0  
 3                  0  
 6                  0  ,
 'Survived_<item>_1':    Sex_<item>_F  Sex_<item>_M  Age_<item>_O  Age_<item>_Y  Class_<item>_1  \
 0             0             1             0             1               1   
 1             1             0             0             1               

### Target States

In [10]:
unwanted_state = target + '_<item>_' + str(wanted_change_in_target[0])
wanted_state = target + '_<item>_' + str(wanted_change_in_target[1])

# APRIORI
### Generates candidates
Generates canidates and also mines classification rules

In [11]:
action_rules = []
classification_rules = defaultdict(lambda: {'wanted': [], 'unwanted': []})
    

def generate_candidates(attribute_prefix, itemset_prefix, stable_items_binding, flexible_items_binding, unwanted_mask, wanted_mask, actionable_attributes=0):
    K = len(itemset_prefix) + 1
    reduced_stable_items_binding = stable_items_binding.copy()
    reduced_flexible_items_binding = flexible_items_binding.copy()
    new_prefixes = []

    number_of_stable_attributes = len(stable_items_binding) - (min_stable_attributes - K)
    
    if unwanted_mask is None:
        unwanted_frame = frames[unwanted_state]
        wanted_frame = frames[wanted_state]
    else:
        #unwanted_frame = frames[unwanted_state] * unwanted_mask
        unwanted_frame = frames[unwanted_state].multiply(unwanted_mask, axis="index")
        #wanted_frame = frames[wanted_state] * wanted_mask
        wanted_frame = frames[wanted_state].multiply(wanted_mask, axis="index")
    
    for i, (attribute, items) in enumerate(stable_items_binding.items()):
        if i == number_of_stable_attributes:
            break
        for item in items.copy():
            
            break_loop = False
            cand_prefix = itemset_prefix + (item, )
            for stop in stop_list:
                start_comparison = len(cand_prefix)-len(stop)
                if start_comparison >=0:
                    if cand_prefix[start_comparison:] == stop:
                        break_loop = True
                        break
            if break_loop:
                continue
            print('SUPPORT')
            print(cand_prefix)
            
            unwanted_support = unwanted_frame[item].sum()
            wanted_support = wanted_frame[item].sum()
            if unwanted_support < min_unwanted_support or wanted_support < min_wanted_support:
                reduced_stable_items_binding[attribute].remove(item)
            else:
                new_prefixes.append(item)
            
    if K > min_stable_attributes:

        number_of_flexible_attributes = len(flexible_items_binding) - (min_flexible_attributes - actionable_attributes)

        for i, (attribute, items) in enumerate(flexible_items_binding.items()):  
            
            unwanted_states = []
            wanted_states = []
            for item in items:
                
                break_loop = False
                cand_prefix = itemset_prefix + (item, )
                for stop in stop_list:
                    start_comparison = len(cand_prefix)-len(stop)
                    if start_comparison >=0:
                        if cand_prefix[start_comparison:] == stop:
                            break_loop = True
                            break
                if break_loop:
                    continue
                print('SUPPORT')
                print(cand_prefix)
                
                unwanted_support = unwanted_frame[item].sum()
                wanted_support = wanted_frame[item].sum()
                # is unwanted
                if wanted_support + unwanted_support == 0:
                    unwanted_conf = 0
                else:
                    unwanted_conf = unwanted_support/(wanted_support + unwanted_support)
                if unwanted_support >= min_unwanted_support and unwanted_conf >= min_unwanted_confidence:
                    unwanted_states.append({'item': item, 'support': unwanted_support, 'confidence':unwanted_conf})
                # is wanted
                wanted_conf = wanted_support/(wanted_support + unwanted_support)   
                if wanted_support >= min_wanted_support and wanted_conf >= min_wanted_confidence:
                    wanted_states.append({'item': item, 'support': wanted_support, 'confidence': wanted_conf})    
            if actionable_attributes == 0 and (len(unwanted_states) == 0 or len(wanted_states) == 0):
                del reduced_flexible_items_binding[attribute]
                for stop_item in items:
                    stop_list.append(itemset_prefix + (stop_item, ))
            else:
                if i >= number_of_flexible_attributes:
                    new_prefixes.append(item)
                if actionable_attributes + 1 >= min_flexible_attributes:
                    new_attribute_prefix = attribute_prefix + (attribute, )
                    for unwanted_item in unwanted_states:
                        new_itemset_prefix = itemset_prefix + (unwanted_item['item'], )
                        classification_rules[new_attribute_prefix]['unwanted'].append({
                                             'itemset': new_itemset_prefix,  
                                             'support': unwanted_item['support'],
                                             'confidence': unwanted_item['confidence'],
                                             'target': wanted_change_in_target[0]
                                            })
                    for wanted_item in wanted_states:
                        new_itemset_prefix = itemset_prefix + (wanted_item['item'], )
                        classification_rules[new_attribute_prefix]['wanted'].append({
                                             'itemset':new_itemset_prefix, 
                                             'support': wanted_item['support'],
                                             'confidence': wanted_item['confidence'],
                                             'target': wanted_change_in_target[1]
                                            })
    
    new_candidates = []
    for new_prefix in new_prefixes:
        is_in_tree = False
        new_stable_items_binding = defaultdict(lambda: [])
        new_flexible_items_binding = defaultdict(lambda: [])
        new_actionable_attributes = actionable_attributes
        
        for attribute, items in reduced_stable_items_binding.items():
            for item in items:
                if is_in_tree:
                    new_stable_items_binding[attribute].append(item)
                if item == new_prefix:
                    is_in_tree = True
        for attribute, items in reduced_flexible_items_binding.items():
            for item in items:
                if is_in_tree:
                    new_flexible_items_binding[attribute].append(item)
                if item == new_prefix:
                    is_in_tree = True
                    new_actionable_attributes += 1
        new_itemset_prefix = itemset_prefix + (new_prefix,)
        new_attribute_prefix = attribute_prefix + (new_prefix[:new_prefix.index('_<item>_')], )
        new_candidates.append({
                               'attribute_prefix': new_attribute_prefix,
                               'itemset_prefix': new_itemset_prefix,
                               'stable_items_binding': new_stable_items_binding,
                               'flexible_items_binding': new_flexible_items_binding,
                               'unwanted_mask': unwanted_frame[new_prefix],
                               'wanted_mask': wanted_frame[new_prefix],
                               'actionable_attributes': new_actionable_attributes,
                              })
    return new_candidates



### Generates Action Rules

In [12]:
def generate_action_rules():
    for attribute_prefix, rules in classification_rules.items():
        for wanted_rule in rules['wanted']:
            for unwanted_rule in rules['unwanted']:
                action_rules.append({'unwanted': unwanted_rule, 'wanted': wanted_rule})

### Apriori iterations

In [13]:
def ar_apriori():
    candidates_queue = [{
                         'attribute_prefix': tuple(),
                         'itemset_prefix':tuple(), 
                         'stable_items_binding': stable_items_binding, 
                         'flexible_items_binding': flexible_items_binding,
                         'unwanted_mask': None,
                         'wanted_mask': None,
                         'actionable_attributes':0
                        }]
    while len(candidates_queue)>0:
        candidate = candidates_queue.pop(0)
        new_candidates = generate_candidates(**candidate)
        candidates_queue += new_candidates
    generate_action_rules()

print('ITEMSET - counting support')
ar_apriori()
print('')
print('ACTION RULES')
print(action_rules)

ITEMSET - counting support
SUPPORT
('Sex_<item>_F',)
SUPPORT
('Sex_<item>_M',)
SUPPORT
('Sex_<item>_F', 'Age_<item>_O')
SUPPORT
('Sex_<item>_F', 'Age_<item>_Y')
SUPPORT
('Sex_<item>_M', 'Age_<item>_O')
SUPPORT
('Sex_<item>_M', 'Age_<item>_Y')
SUPPORT
('Sex_<item>_F', 'Age_<item>_Y', 'Class_<item>_1')
SUPPORT
('Sex_<item>_F', 'Age_<item>_Y', 'Class_<item>_2')
SUPPORT
('Sex_<item>_F', 'Age_<item>_Y', 'Embarked_<item>_C')
SUPPORT
('Sex_<item>_F', 'Age_<item>_Y', 'Embarked_<item>_S')

ACTION RULES
[{'unwanted': {'itemset': ('Sex_<item>_F', 'Age_<item>_Y', 'Class_<item>_2'), 'support': 1, 'confidence': 1.0, 'target': 0}, 'wanted': {'itemset': ('Sex_<item>_F', 'Age_<item>_Y', 'Class_<item>_1'), 'support': 2, 'confidence': 1.0, 'target': 1}}]
